In [1]:
#default_exp psm_reader.psm_reader

In [2]:
#hide
%reload_ext autoreload
%autoreload 2

In [3]:
#export
import typing
import numpy as np
import pandas as pd

from alphabase.peptide.fragment import get_charged_frag_types

def translate_other_modification(
    mod_str: str, 
    mod_dict: dict
)->str:
    '''
    Translate modifications in `mod_str` to the other 
    format mapped by mod_dict.
    Args:
        mod_str (str): mod list in str format, seperated by ';', 
            e.g. ModA;ModB
        mod_dict (dict): translate mod dict from others to AlphaBase, 
            e.g. for pFind, key='Phospho[S]', value='Phospho@S'
    Returns:
        str: new mod list in str format seperated by ';' if all
             modifications are in `mod_dict` else pd.NA.
    '''
    if not mod_str: return ""
    ret_mods = []
    for mod in mod_str.split(';'):
        if mod in mod_dict:
            ret_mods.append(mod_dict[mod])
        else:
            return pd.NA
    return ";".join(ret_mods)

def keep_modifications(
    mod_str: str, 
    mod_set: set
)->str:
    '''
    Check if modifications in `mod_str` in `mod_set`
    Args:
        mod_str (str): mod list in str format, seperated by ';', 
            e.g. Oxidation@M;Phospho@S.
        mod_set (set): mod set to check
    Returns:
        str: `mod_str` if all modifications are in mod_set 
             else pd.NA.
    '''
    if not mod_str: return ""
    for mod in mod_str.split(';'):
        if not mod in mod_set:
            return pd.NA
    return mod_str


class PSMReaderBase(object):
    def __init__(self, 
    ):
        # modification_convert_dict=dict[str, str]: 
        #     key:   mod names of other search engines
        #     value: mod names in AlphaBase
        # It is used to convert mods of other engines 
        # to AlphaBase format. Different search engines
        # have different mod names.

        self.modification_convert_dict = {}

        self.column_mapping = {
            'sequence': 'NakedSequence',
            # AlphaBase does not need 'modified_sequence',
            # but it will get 'mods', 'mod_sites' from it
            'modified_sequence': 'ModifiedSequence',
            'charge': 'Charge',
            # If the value is a list, check if one of the columns exist
            # and get 'proteins' from that column
            'proteins':['Proteins','UniprotIDs'],
            'uniprot_ids':'UniprotIds',
            # Similar to 'proteins'
            'genes': ['Genes','Gene Names','Gene names'],
        } # Add more columns for sub-classes of different tasks

        self._psm_df:pd.DataFrame = None
        self.keep_all_psm = False
        
    @property
    def psm_df(self):
        return self._psm_df

    def _load_file(self, filename:str)->pd.DataFrame:
        """
        Load original dataframe from PSM filename. 
        Different search engines may store PSMs in different ways:
        tsv, csv, HDF, XML, ...

        Args:
            filename (str): psm filename

        Raises:
            NotImplementedError: Sub-classes must re-implement this method

        Returns:
            pd.DataFrame: dataframe loaded
        """
        raise NotImplementedError(
            f'"{self.__class__}" must re-implement "_load_file()"'
        )

    def _translate_columns(self, origin_df:pd.DataFrame):
        """
        Translate the dataframe from other search engines 
        to AlphaBase format

        Args:
            origin_df (pd.DataFrame): df of other search engines
        """
        self._psm_df = pd.DataFrame()
        for col, map_col in self.column_mapping.items():
            if isinstance(map_col, str):
                if map_col in origin_df.columns:
                    self._psm_df[col] = origin_df[map_col]
                else:
                    self._psm_df[col] = pd.NA
            else:
                for other_col in map_col:
                    if other_col in origin_df.columns:
                        self._psm_df[col] = origin_df[other_col]
                        break
                if col not in self._psm_df.columns:
                    self._psm_df[col] = pd.NA
        origin_df['nAA'] = origin_df[self.column_mapping['sequence']].str.len()
        self._psm_df['nAA'] = origin_df['nAA']

    def _translate_modifications(self):
        '''
        Translate modifications to AlphaBase format.

        Raises: KeyError if `mod` in `mod_names` is 
            not in `self.modification_convert_dict`
        '''
        self._psm_df.mods = self._psm_df.mods.apply(
            translate_other_modification, 
            mod_dict=self.modification_convert_dict
        )

    def _post_process(self, 
        filename:str, origin_df:pd.DataFrame
    ):
        """
        Remove unknown modifications and perform other post processings, 
        e.g. loading fragments for AlphaQuant or AlphaDeep

        Args:
            filename (str): psm filename
            origin_df (pd.DataFrame): the loaded original df
        """
        origin_df = origin_df[
            ~self._psm_df['mods'].isna()
        ].reset_index(drop=True)
        
        self._psm_df = self._psm_df[
            ~self._psm_df['mods'].isna()
        ].reset_index(drop=True)

    def load(self, filename):
        origin_df = self._load_file(filename)
        self._translate_columns(origin_df)
        self._translate_modifications()
        self._post_process(filename, origin_df)

    def filter_psm_by_modifications(self, include_mod_list = [
        'Oxidation@M','Phospho@S','Phospho@T','Phospho@Y','Acetyl@Protein N-term'
    ]):
        '''
            Only keeps peptides with modifications in `include_mod_list`.
        '''
        mod_set = set(include_mod_list)
        self._psm_df.mods = self._psm_df.mods.apply(keep_modifications, mod_set=mod_set)
        
        self._psm_df.dropna(
            subset=['mods'], inplace=True
        )
        self._psm_df.reset_index(drop=True, inplace=True)

class PSMReader_w_FragBase(PSMReaderBase):
    '''
    Read PSMs and fragments
    '''
    def __init__(self,
        frag_types=['b','y','b_modloss','y_modloss'], 
        max_frag_charge=2,
        frag_tol=20, frag_ppm=True,
    ):
        super().__init__()

        self.charged_frag_types = get_charged_frag_types(
            frag_types, max_frag_charge
        )
        self._fragment_intensity_df:pd.DataFrame = pd.DataFrame(
            columns=self.charged_frag_types
        )

        self.frag_tol = frag_tol
        self.frag_ppm = frag_ppm
    
    @property
    def fragment_intensity_df(self):
        return self._fragment_intensity_df


In [4]:
#export
class PSMReaderProvider:
    def __init__(self):
        self.reader_dict = {}

    def register_reader(self, reader_name, reader_class):
        # for example, we can register the MSFragger reader 
        self.reader_dict[reader_name.lower()] = reader_class

    def get_reader(self, reader_name, 
    )->PSMReaderBase:
        return self.reader_dict[reader_name.lower()]()

psm_reader_provider = PSMReaderProvider()

In [5]:
#export
class PSMwFragReaderProvider:
    def __init__(self):
        self.reader_dict = {}

    def register_reader(self, reader_name, reader_class):
        # for example, we can register the AlphaPept reader 
        self.reader_dict[reader_name.lower()] = reader_class

    def get_reader(self, reader_name, 
        frag_types=['b','y','b_modloss','y_modloss'], 
        max_frag_charge=2,
        frag_tol=20, frag_ppm=True,
    )->PSMReader_w_FragBase:
        return self.reader_dict[reader_name.lower()](
            frag_types, max_frag_charge, 
            frag_tol, frag_ppm,
        )

psm_w_frag_reader_provider = PSMwFragReaderProvider()

In [6]:
#hide
import io
import alphadeep.psm_reader.pfind_reader #to register reader
from alphadeep.psm_reader.psm_reader import psm_reader_provider
psmlabel_str = '''spec	peptide	modinfo	b	b-NH3	b-H2O	b-ModLoss	y	y-NH3	y-H2O	y-ModLoss
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.31809.31809.2.0.dta	PSTDLLMLK	2,Phospho[S];7,Oxidation[M];	b2+1,11394796;b3+1,1242152.8;b4+1,3736963.3;b4+2,169730.9;b5+1,1963146.4;b6+1,1264694.9;b6+2,265013.9;b7+1,1253226.5;b7+2,909294.6;b8+1,720161.7;		b2-H2O+1,1392711.1;b3-H2O+1,2807275.5;b4-H2O+1,656366;b5-H2O+1,341585;b6-H2O+1,209442.1;	b7-ModLoss+1,473386.4;b8-ModLoss+1,208994.1;	y8+1,22006548;y8+2,256042.3;y7+1,19231634;y7+2,213004.9;y6+1,6696723;y5+1,5890172;y4+1,4885660.5;y3+1,3570823.5;y2+1,1857323.8;y1+1,1636183.8;	y8-NH3+1,567207.4;y1-NH3+1,531551.1;	y8-H2O+1,1416820.1;y8-H2O+2,256081;y7-H2O+1,900931.1;y7-H2O+2,2961118.5;y3-H2O+1,184890.4;y2-H2O+1,306988.6;y1-H2O+1,1126237.5;	y8-ModLoss+1,4600049;y7-ModLoss+1,3840026.3;y6-ModLoss+1,1045096.9;y5-ModLoss+1,868705.3;y4-ModLoss+1,573257.7;y3-ModLoss+1,518627;
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.23862.23862.2.0.dta	HTAYSDFLSDK		b1+1,299364.8;b2+1,3488062;b3+1,308160.7;b4+1,233294.5;b5+1,55810.8;b6+1,650653.9;b7+1,485245;b8+1,328604.8;b9+1,160565.1;b10+1,376348.6;	b7-NH3+1,63030.5;b10-NH3+1,129601.2;	b2-H2O+1,176123.1;b3-H2O+1,114956.5;b4-H2O+1,59385.5;b5-H2O+1,41324.8;b6-H2O+1,527812.9;b7-H2O+1,275831.8;b8-H2O+1,365457.2;b9-H2O+1,227540.1;b9-H2O+2,59055.5;b10-H2O+1,265041.1;b10-H2O+2,55810.8;		y10+1,2513661;y9+1,3651241.3;y8+1,989975.4;y7+1,594356.4;y6+1,155207.8;y5+1,1266161.9;y4+1,321580;y3+1,1227822.8;y2+1,636557.6;y1+1,697604.3;	y10-NH3+1,75562.7;y7-NH3+1,102006.4;y1-NH3+1,185766.1;	y10-H2O+1,189888.1;y9-H2O+1,73236.7;y4-H2O+1,56329.2;y3-H2O+1,91522.7;y2-H2O+1,98231.2;y1-H2O+1,375849.7;	
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.23431.23431.2.0.dta	HTAYSDFLSDK		b1+1,45976.2;b2+1,568759.5;b3+1,49093.1;b4+1,49601;b5+1,23729.4;b6+1,141218;b7+1,104082.9;b8+1,115693.4;b9+1,60744.1;b10+1,98634.1;	b5-NH3+1,12496.8;b8-NH3+1,33514.1;b9-NH3+1,34818.7;	b2-H2O+1,13616.9;b3-H2O+1,9902.4;b4-H2O+1,29442.6;b5-H2O+1,13391.7;b6-H2O+1,54826.9;b7-H2O+1,62953.9;b8-H2O+1,69100.3;b9-H2O+1,60146.4;b10-H2O+1,50907.2;b10-H2O+2,23729.4;		y10+1,361255.9;y9+1,552602.6;y8+1,160028.2;y7+1,102606.7;y6+1,22479.1;y5+1,167033.7;y4+1,76430.6;y3+1,273281.6;y2+1,165234.1;y1+1,142589;	y7-NH3+1,22439.1;y1-NH3+1,37364.8;	y10-H2O+1,29709;y9-H2O+1,16514.8;y3-H2O+1,36499.1;y2-H2O+1,17987.4;y1-H2O+1,96955.6;	
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.32733.32733.2.0.dta	HFALFSTDVTK		b1+1,27135.7;b2+1,361137.4;b3+1,68835.3;b4+1,70138.3;b5+1,45754.8;b7+1,11576.6;b8+1,91503.8;b9+1,64331.7;b10+1,27626.7;b10+2,25667;		b3-H2O+1,48033;b9-H2O+1,14316.2;b10-H2O+1,11975.8;		y10+1,219460.2;y10+2,13433.4;y9+1,442455.6;y8+1,97392.2;y7+1,108960.5;y6+1,60849.7;y5+1,26771.3;y4+1,17036.4;y3+1,45523.9;y2+1,103608.1;y1+1,62643;	y6-NH3+2,11445.5;y1-NH3+1,18111.4;	y2-H2O+1,15362.3;y1-H2O+1,34004.8;	
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.23669.23669.2.0.dta	HTAYSDFLSDK		b1+1,262855;b2+1,3235572.3;b3+1,268667.7;b4+1,237506.8;b5+1,80077.3;b6+1,557696.8;b7+1,336325.9;b7+2,31299.9;b8+1,247175;b8+2,28601.6;b9+1,116897.4;b9+2,18714.8;b10+1,275498.9;	b2-NH3+1,19037.2;	b2-H2O+1,141344.2;b3-H2O+1,92893.6;b4-H2O+1,56392;b5-H2O+1,46386.1;b6-H2O+1,404526;b7-H2O+1,203047.2;b7-H2O+2,13485.6;b8-H2O+1,231333.9;b8-H2O+2,30468.7;b9-H2O+1,151952.4;b9-H2O+2,53914;b10-H2O+1,172398.7;b10-H2O+2,80077.3;		y10+1,1652851.5;y10+2,31706.2;y9+1,2379192.5;y8+1,664060.9;y8+2,26944.2;y7+1,418105.1;y6+1,118890.7;y5+1,1026599.5;y4+1,309265.2;y3+1,1084321;y2+1,608127.8;y1+1,617369.5;	y10-NH3+1,41452.9;y7-NH3+1,61761.1;y2-NH3+1,32386.8;y1-NH3+1,199112.3;	y10-H2O+1,127643.4;y9-H2O+1,49576.6;y8-H2O+1,26233.2;y6-H2O+1,13648.5;y5-H2O+1,34467.8;y4-H2O+1,28410.1;y3-H2O+1,75421.2;y2-H2O+1,106013.4;y1-H2O+1,351150.3;	
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.32408.32408.2.0.dta	HFALFSTDVTK		b1+1,39174;b2+1,547471.8;b3+1,97899.3;b4+1,102380.5;b5+1,64629.1;b6+1,18020.3;b6+2,11095.4;b8+1,102782.6;b9+1,70052.5;b10+1,32341.3;b10+2,19485.5;	b2-NH3+1,17124.9;	b3-H2O+1,81865.9;b8-H2O+1,16527.8;b10-H2O+1,22699.3;		y10+1,251072.3;y9+1,579795.4;y8+1,126733.6;y7+1,154207.1;y6+1,73626;y5+1,19896.1;y4+1,20851.9;y3+1,70220.2;y2+1,127975.4;y1+1,81733.5;	y1-NH3+1,23140.9;	y2-H2O+1,18157.2;y1-H2O+1,43263.3;	
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.31708.31708.2.0.dta	PSTDLLMLK	7,Oxidation[M];	b2+1,15111598;b3+1,1437139.1;b4+1,4843799.5;b4+2,308051.3;b5+1,2508311;b6+1,1812935.6;b6+2,332253.3;b7+1,1607183.6;b7+2,1064015.8;b8+1,1010607.1;		b2-H2O+1,1739067.1;b3-H2O+1,3721898;b4-H2O+1,832363.4;b5-H2O+1,598314.4;b6-H2O+1,370911.8;b7-H2O+1,342069.2;b8-H2O+1,267002.9;	b7-ModLoss+1,667185.4;b8-ModLoss+1,233204;	y8+1,28281878;y8+2,367999.1;y7+1,22845608;y7+2,591054.7;y6+1,8346688;y5+1,6896273;y5+2,145534.7;y4+1,5982969;y3+1,4114733.5;y2+1,2586525.3;y1+1,2430101.8;	y7-NH3+1,376892.8;y6-NH3+1,217853;y1-NH3+1,732089.9;	y8-H2O+1,1979115.4;y8-H2O+2,324478.8;y7-H2O+1,1182049.3;y7-H2O+2,4076522;y6-H2O+1,180659.8;y5-H2O+1,166679.8;y3-H2O+1,163515.6;y2-H2O+1,357436.9;y1-H2O+1,1743094.4;	y8-ModLoss+1,6069443.5;y7-ModLoss+1,4348249;y6-ModLoss+1,1384640.9;y5-ModLoss+1,1112314.8;y4-ModLoss+1,563618.3;y3-ModLoss+1,625598.4;
01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1.21837.21837.2.0.dta	HDLDYGIDSYK		b1+1,41983.3;b2+1,379322.4;b3+1,56831.5;b4+1,115561.6;b5+1,42836.4;b6+1,50857.8;b7+1,51406.2;b7+2,18378.2;b8+1,113548.1;b8+2,9950.8;b9+1,59166.4;b9+2,15431;b10+1,48784.2;	b10-NH3+1,13714.9;	b2-H2O+1,13046.2;b3-H2O+1,64792.4;b4-H2O+1,11627.2;b9-H2O+1,41938.1;b10-H2O+1,26039.3;		y10+1,305846.1;y9+1,262227.6;y8+1,113308;y7+1,140019.9;y6+1,74818.5;y4+1,50029.2;y3+1,135117.1;y2+1,55729.5;y1+1,93491.5;	y10-NH3+1,15723.9;y9-NH3+1,10144.8;y1-NH3+1,28375.2;	y7-H2O+1,11441.7;y1-H2O+1,42537.6;	
'''
reader = psm_reader_provider.get_reader('psmlabel')
reader.load(io.StringIO(psmlabel_str))
reader.psm_df

,sequence,charge,rt,ccs,raw_name,query_id,spec_idx,nAA,mods,mod_sites,frag_start_idx,frag_end_idx
0,PSTDLLMLK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,31809,9,Phospho@S;Oxidation@M,2;7,0,8
1,HTAYSDFLSDK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,23862,11,,,8,18
2,HTAYSDFLSDK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,23431,11,,,18,28
3,HFALFSTDVTK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,32733,11,,,28,38
4,HTAYSDFLSDK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,23669,11,,,38,48
5,HFALFSTDVTK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,32408,11,,,48,58
6,PSTDLLMLK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,31708,9,Oxidation@M,7,58,66
7,HDLDYGIDSYK,2,<NA>,<NA>,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,21837,11,,,66,76


In [7]:
#hide
from alphabase.spectrum_library.library_base import SpecLibBase

lib = SpecLibBase([])
lib._precursor_df = reader.psm_df.fillna(np.nan)
#lib._precursor_df = reader.psm_df['RT'].fillna(np.nan)
lib._fragment_mz_df = reader._fragment_intensity_df
lib._fragment_intensity_df = reader._fragment_intensity_df
lib.save_hdf('../../sandbox/lib.hdf')

In [8]:
#hide
lib = SpecLibBase([])
lib.load_hdf('../../sandbox/lib.hdf')
lib.precursor_df

,ccs,charge,frag_end_idx,frag_start_idx,mod_sites,mods,nAA,query_id,raw_name,rt,sequence,spec_idx
0,NaN,2,8,0,2;7,Phospho@S;Oxidation@M,9,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,PSTDLLMLK,31809
1,NaN,2,18,8,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HTAYSDFLSDK,23862
2,NaN,2,28,18,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HTAYSDFLSDK,23431
3,NaN,2,38,28,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HFALFSTDVTK,32733
4,NaN,2,48,38,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HTAYSDFLSDK,23669
5,NaN,2,58,48,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HFALFSTDVTK,32408
6,NaN,2,66,58,7,Oxidation@M,9,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,PSTDLLMLK,31708
7,NaN,2,76,66,,,11,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1...,01625b_GA3-TUM_first_pool_17_01_01-3xHCD-1h-R1,NaN,HDLDYGIDSYK,21837


In [9]:
#hide
lib.fragment_mz_df

,b_modloss_z1,b_modloss_z2,b_z1,b_z2,y_modloss_z1,y_modloss_z2,y_z1,y_z2
0,0.0,0.0,0.000000,0.000000,0.209031,0.0,1.000000,0.011635
1,0.0,0.0,0.517791,0.000000,0.174495,0.0,0.873905,0.009679
2,0.0,0.0,0.056445,0.000000,0.047490,0.0,0.304306,0.000000
3,0.0,0.0,0.169811,0.007713,0.039475,0.0,0.267655,0.000000
4,0.0,0.0,0.089207,0.000000,0.026049,0.0,0.222009,0.000000
...,...,...,...,...,...,...,...,...
71,0.0,0.0,0.134075,0.000000,0.000000,0.0,0.000000,0.000000
72,0.0,0.0,0.135521,0.048450,0.000000,0.0,0.131891,0.000000
73,0.0,0.0,0.299345,0.026233,0.000000,0.0,0.356206,0.000000
74,0.0,0.0,0.155979,0.040680,0.000000,0.0,0.146919,0.000000


In [10]:
#hide
lib.fragment_intensity_df

,b_modloss_z1,b_modloss_z2,b_z1,b_z2,y_modloss_z1,y_modloss_z2,y_z1,y_z2
0,0.0,0.0,0.000000,0.000000,0.209031,0.0,1.000000,0.011635
1,0.0,0.0,0.517791,0.000000,0.174495,0.0,0.873905,0.009679
2,0.0,0.0,0.056445,0.000000,0.047490,0.0,0.304306,0.000000
3,0.0,0.0,0.169811,0.007713,0.039475,0.0,0.267655,0.000000
4,0.0,0.0,0.089207,0.000000,0.026049,0.0,0.222009,0.000000
...,...,...,...,...,...,...,...,...
71,0.0,0.0,0.134075,0.000000,0.000000,0.0,0.000000,0.000000
72,0.0,0.0,0.135521,0.048450,0.000000,0.0,0.131891,0.000000
73,0.0,0.0,0.299345,0.026233,0.000000,0.0,0.356206,0.000000
74,0.0,0.0,0.155979,0.040680,0.000000,0.0,0.146919,0.000000
